<a href="https://colab.research.google.com/github/AwkwaBear/EE-628/blob/main/sg13g2_LDO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Design and Optimization of Analog LDO Using sg13g2 PDK with Relational Graph Neural Network and Reinforcement Learning


|Name|
|:--:|
|Vikas Shukla |
|Zion McDowell|
|Jie Zhao|
|Anthony Gasbarro|


### Install dependencies

- sg13g2 [Process Development Kit]
- Ngspice [Circuit Simulation Tool]
- Conda [Python Development Virtual Environment]
- Pytorch [Machine Learning Framework]
- PyG [Graph Neural Network Library]
- Tabulate [Data Table Export Interface Library]

Note: that after the installation of these packages, **you may need to restart the runtime to use these newly installed packages**.

In [1]:
import os

# install conda.
CONDA_PREFIX = os.environ.get('CONDA_PREFIX', None)
if not CONDA_PREFIX:
  !python -m pip install condacolab
  import condacolab
  condacolab.install()

# install sg13g2
#TODO: Verify proper syntax for sg13g2 PDK
!conda install -c litex-hub open_pdks.sg13g2

# install ngspice
!conda install -c conda-forge ngspice

# Install Pytorch with cpu-only option.
!conda install pytorch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1 cpuonly -c pytorch

# Check the version of Python (should be larger than 3.7) and Pytorch.
import torch
import sys
print("Python version")
print (sys.version)

!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"

# Install PyG to enable graph neural networks. We need to manually guide Colab to install the correct version of PyG based on which Pytorch version we have installed
# (https://colab.research.google.com/drive/17VFsRZeZa7rgtrQ39-RMiYIE0ePrSlgN?authuser=1#scrollTo=rFwf_0DrvXLv). In this case it is 1.13.1 with cpu-only option.
!pip install torch-scatter -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-sparse -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-cluster -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.13.1+cpu.html
!pip install torch-geometric

# Check the version of PyG.
import torch_geometric
print(torch_geometric.__version__)

# Install Gymnasium for the RL environment.
!pip install gymnasium

# Check Gymnasium version.
import gymnasium as gym
print(gym.__version__)

# Install tabulate for showing simulation results nicely during the runtime later.
!pip install tabulate

# Similarly, check tabulate version.
import tabulate
print(tabulate.__version__)

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.1.0-1/Mambaforge-23.1.0-1-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:14
🔁 Restarting kernel...
Solving environment: | failed with initial frozen solve. Retrying with flexible solve.
Solving environment: / failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - open_pdks.sg13g2

Current channels:

  - https://conda.anaconda.org/litex-hub/linux-64
  - https://conda.anaconda.org/litex-hub/noarch
  - https://conda.anaconda.org/conda-forge/linux-64
  - https://conda.anaconda.org/conda-forge/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.


Solving environment: - \ | / - \ | / - \ | / - done


==

### Setting up g13g2 circuit simulation environment


Create some directory to organize files.

In [1]:
!mkdir g13g2_ldo
!mkdir g13g2_ldo/simulations
!mkdir g13g2_ldo/saved_weights
!mkdir g13g2_ldo/saved_memories

Setting up Ngspice initialization for g13g2. Also do use 8 threads to speed up simulations.

In [2]:
%%writefile /content/g13g2_ldo/simulations/.spiceinit
* ngspice initialization for g13g2
* assert BSIM compatibility mode with "nf" vs. "W"
set ngbehavior=hsa
* "nomodcheck" speeds up loading time
set ng_nomodcheck
set num_threads=8

Writing /content/g13g2_ldo/simulations/.spiceinit


Write LDO netlist. Noticed that it calls the analysis and variables of the netlist from another two spice scripts (`ldo_tb_analysis.spice` and `ldo_tb_vars.spice`). We will include them later.



In [5]:
!conda install -c litex-hub open_pdks.sg13g2

/ - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
Solvi

In [4]:
#TODO: MODIFY BELOW CODE FOR G13g2
%%writefile /content/sky130_ldo/simulations/ldo_tb.spice
** sch_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/ldo_tb.sch
**.subckt ldo_tb Vreg Vreg1
*.opin Vreg
*.opin Vreg1
Vref Vref GND Vref
.save i(vref)
Vb net1 GND Vb
.save i(vb)
IL Vreg net2 dc IL PULSE(10u IL 0 10n 10n 50u 100u 0)
Vdd Vdd GND ac 1 dc Vdd
.save i(vdd)
x1 Vdd Vreg Vref net1 GND Vreg ldo
XCL Vreg GND sky130_fd_pr__cap_mim_m3_1 W=30 L=30 MF=M_CL m=M_CL
Rdummy net2 GND 1 m=1
Vref1 Vref1 GND Vref
.save i(vref1)
Vb1 net3 GND Vb
.save i(vb1)
IL1 Vreg1 GND IL
Vdd1 Vdd1 GND Vdd
.save i(vdd1)
x2 Vdd1 net4 Vref1 net3 GND Vreg1 ldo
XCL1 Vreg1 GND sky130_fd_pr__cap_mim_m3_1 W=30 L=30 MF=M_CL m=M_CL
Vprobe2 probe net4 dc 0
.save i(vprobe2)
Vprobe1 probe Vreg1 dc 0 ac 1
.save i(vprobe1)
Iprobe1 GND probe dc 0 ac 0
**** begin user architecture code

* Not loading the entire design corner which takes too long
*.lib /usr/local/share/pdk/sky130A/libs.tech/ngspice/sky130.lib.spice tt

* Instead just load tt corner which is faster
.param mc_mm_switch=0
.param mc_pr_switch=0
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/corners/tt.spice
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/r+c/res_typical__cap_typical.spice
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/r+c/res_typical__cap_typical__lin.spice
.include /usr/local/share/pdk/sky130A/libs.tech/ngspice/corners/tt/specialized_cells.spice

.control
* save all voltage and current
save all
.options savecurrents
set filetype=ascii
set units=degrees

.include /content/sky130_ldo/simulations/ldo_tb_analysis.spice
.endc

.include /content/sky130_ldo/simulations/ldo_tb_vars.spice

**** end user architecture code
**.ends

* expanding   symbol:  ldo.sym # of pins=6
** sym_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/ldo.sym
** sch_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/ldo.sch
.subckt ldo Vdd Vfb Vref Vb Vss Vreg
*.iopin Vb
*.iopin Vss
*.opin Vreg
*.iopin Vref
*.ipin Vdd
*.iopin Vfb
x1 Vdd net1 Vfb Vref Vb Vss diff_pair
XM6 Vreg net1 Vdd Vdd sky130_fd_pr__pfet_g5v0d10v5 L=L_pass W=W_pass nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=M_pass m=M_pass
XCfb net2 Vreg sky130_fd_pr__cap_mim_m3_1 W=10 L=10 MF=M_Cfb m=M_Cfb
XRfb net2 net1 Vss sky130_fd_pr__res_high_po_0p35 L=3 mult=M_Rfb m=M_Rfb
.ends


* expanding   symbol:  diff_pair.sym # of pins=6
** sym_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/diff_pair.sym
** sch_path: /fs1/eecg/tcc/lizongh2/sky130_ldo/xschem/diff_pair.sch
.subckt diff_pair Vdd vout vinp vinm Vb Vss
*.iopin Vdd
*.ipin vinm
*.ipin vinp
*.iopin Vb
*.opin vout
*.iopin Vss
XM1 net1 vinp net2 Vss sky130_fd_pr__nfet_g5v0d10v5 L=L_M1 W=W_M1 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM2 vout vinm net2 Vss sky130_fd_pr__nfet_g5v0d10v5 L=L_M2 W=W_M2 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM3 vout net1 Vdd Vdd sky130_fd_pr__pfet_g5v0d10v5 L=L_M3 W=W_M3 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM4 net1 net1 Vdd Vdd sky130_fd_pr__pfet_g5v0d10v5 L=L_M4 W=W_M4 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
XM5 net2 Vb Vss Vss sky130_fd_pr__nfet_g5v0d10v5 L=L_M5 W=W_M5 nf=1 ad='int((nf+1)/2) * W/nf * 0.29'
+ as='int((nf+2)/2) * W/nf * 0.29' pd='2*int((nf+1)/2) * (W/nf + 0.29)' ps='2*int((nf+2)/2) * (W/nf + 0.29)'
+ nrd='0.29 / W' nrs='0.29 / W' sa=0 sb=0 sd=0 mult=1 m=1
.ends

.GLOBAL GND
.end


Writing /content/sky130_ldo/simulations/ldo_tb.spice


FileNotFoundError: [Errno 2] No such file or directory: '/content/sky130_ldo/simulations/ldo_tb.spice'